In [1]:
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from openpilot.selfdrive.car.ford.values import CAR

database = get_comma_car_segments_database()
platforms = [c.value for c in CAR] + ['mock']
segments = []

for platform in platforms:
  if platform not in database:
    print(f'Skipping platform: {platform}, no data available')
    continue

  platform_segments = database[platform]
  print(f'Got {len(platform_segments)} segments for platform {platform}')
  segments += platform_segments

print(f'Got {len(segments)} segments total')

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


Got 287 segments for platform FORD BRONCO SPORT 1ST GEN
Got 137 segments for platform FORD ESCAPE 4TH GEN
Got 1041 segments for platform FORD EXPLORER 6TH GEN
Got 5 segments for platform FORD F-150 14TH GEN
Got 56 segments for platform FORD FOCUS 4TH GEN
Got 637 segments for platform FORD MAVERICK 1ST GEN
Got 3 segments for platform FORD F-150 LIGHTNING 1ST GEN
Got 3 segments for platform FORD MUSTANG MACH-E 1ST GEN
Got 343 segments for platform mock
Got 2512 segments total


In [2]:
from collections import defaultdict
from multiprocessing import Pool

from tqdm import tqdm

from openpilot.tools.lib.logreader import LogReader


def get_car_params(segment):
  lr = LogReader(segment)
  return lr.first('carParams')


def get_car_params_key(CP):
  fw_versions_dict = defaultdict(set)
  for fw in CP.carFw:
    if not fw.logging:
      sub_addr = fw.subAddress if fw.subAddress != 0 else None
      fw_versions_dict[(fw.address, sub_addr)].add(fw.fwVersion)
  fw_versions_dict = { addr: frozenset(fws) for addr, fws in fw_versions_dict.items() }
  return (CP.carVin, frozenset(fw_versions_dict.items()))


CAR_PARAMS_TO_CHECK = []
CAR_PARAMS_KEYS = set()

with Pool(processes=8) as p:
  with tqdm(total=len(segments)) as pbar:
    for CP in p.imap_unordered(get_car_params, segments):
      pbar.update()
      if CP is None:
        continue
      key = get_car_params_key(CP)
      if key in CAR_PARAMS_KEYS:
        continue
      CAR_PARAMS_KEYS.add(key)
      CAR_PARAMS_TO_CHECK.append(CP)

print(f'Got {len(CAR_PARAMS_TO_CHECK)} car params to check')

100%|██████████| 2512/2512 [17:03<00:00,  2.45it/s]

Got 74 car params to check


In [3]:
import pandas as pd

from openpilot.selfdrive.car.fw_versions import build_fw_dict, match_fw_to_car

results = []
for CP in CAR_PARAMS_TO_CHECK:
  vin = CP.carVin
  real_fingerprint, real_fuzzy, source = CP.carFingerprint, CP.fuzzyFingerprint, CP.fingerprintSource
  car_fw = CP.carFw

  exact_match, determined_fingerprint = match_fw_to_car(car_fw, log=False)
  determined_fuzzy = not exact_match

  if len(determined_fingerprint) == 1:
    determined_fingerprint = determined_fingerprint.pop()
  elif len(determined_fingerprint) == 0:
    determined_fingerprint = 'mock'
  else:
    determined_fingerprint = 'multiple'

  fw_dict = build_fw_dict(CP.carFw)
  has_radar = (0x764, None) in fw_dict.keys()

  changed = (real_fingerprint != determined_fingerprint) or (real_fuzzy != determined_fuzzy)
  results.append((vin, has_radar,
                  real_fingerprint, real_fuzzy, determined_fingerprint, determined_fuzzy,
                  changed))

  if changed:
    print(f'{vin=}  {real_fingerprint=:<28}  source={source}  {determined_fingerprint=:<28} ({exact_match=})')
    if determined_fingerprint != real_fingerprint:
      for addr, fw in build_fw_dict(car_fw).items():
        print(f'  addr={hex(addr[0])} {fw=}')
    print()

df = pd.DataFrame(results, columns=['vin', 'has_radar',
                                    'real_fingerprint', 'real_fuzzy', 'determined_fingerprint', 'determined_fuzzy',
                                    'changed'])
print(df.to_string())

/tmp/ipykernel_232053/1701990771.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


vin='00000000000XXXXXX'  real_fingerprint=FORD F-150 14TH GEN           source=fixed  determined_fingerprint=mock                         (exact_match=True)

vin='3FTTW8E3XPRXXXXXX'  real_fingerprint=FORD MAVERICK 1ST GEN         source=fw  determined_fingerprint=mock                         (exact_match=True)
  addr=0x7e0 fw={b'PZ6A-14C204-JE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x730 fw={b'NZ6C-14D003-AL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x760 fw={b'PZ6C-2D053-ED\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x706 fw={b'NZ6T-14F397-AC\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}

vin='3FTTW8E34PRXXXXXX'  real_fingerprint=FORD MAVERICK 1ST GEN         source=fw  determined_fingerprint=mock                         (exact_match=True)
  addr=0x7e0 fw={b'PZ6A-14C204-JE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x730 fw={b'NZ6C-14D003-AL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
  addr=0x760 fw={b'PZ6C-2D053-ED\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x